In [1]:

import os
import sys
current_dir = os.getcwd()
root_dir = os.path.dirname(os.path.dirname(current_dir))
sys.path.append(root_dir + "studentproject\\anomaly_detection\\")

In [2]:
import pandas as pd
import pyarrow as pa
import numpy as np
import matplotlib.pyplot as plt
import copy

import src.data.utils as utils
import src.data.data_treatment as dt
from src.data.spectrum_decomposition.spectrum import SpectrumDecomposition

%load_ext autoreload
%autoreload 2


In [102]:
#MAKE DIRECTIONS GO UP
subset = utils.load_dataset('TL_raw_data_without_duplicates.parquet', False)
subset = subset.sort_values(by=['id', 'timeindex'])
subset = dt.label_direction(subset, 'TL')
subset = dt.make_direction_up(subset, flip_only_position=False) #CAREFUL, you may need to change the parameter flip only position
filename = 'TL_all_data_without_duplicates.parquet'
dt.export_data(subset, filename)

Loaded dataset: raw_data\TL_raw_data_without_duplicates.parquet
6640-101753 ['down'] [True]
6640-101755 ['unknown' 'up'] [nan True]
6640-101813 ['up'] [True]
6640-101814 ['up'] [True]
6640-101831 ['down'] [True]
6640-101874 ['down'] [True]
6640-101875 ['up'] [True]
6640-101952 ['up'] [True]
6640-101988 ['down'] [False]
6640-101992 ['up'] [False]
6640-102009 ['up'] [False]
6640-102028 ['up'] [True]
6640-102053 ['up'] [True]
6640-102092 ['up'] [True]
6640-102117 ['up'] [True]
6640-102118 ['up'] [False]
6640-102119 ['down'] [True]
6640-102120 ['up'] [True]
6640-102121 ['up'] [True]
6640-102122 ['up'] [False]
6640-102123 ['up'] [True]
6640-102138 ['up'] [True]
6640-102139 ['up'] [False]
6640-102140 ['up'] [False]
6640-102148 ['up'] [True]
6640-102149 ['up'] [True]
6640-102150 ['down'] [True]
6640-102151 ['up'] [True]
6640-102152 ['up'] [False]
6640-102263 ['up'] [False]
6640-102423 ['down'] [True]
6640-102453 ['down'] [False]
6640-102557 ['up'] [True]
6640-102685 ['down'] [False]
6640-1027

In [103]:
#CHECK IF THE NEW DIRECTIONS ARE CORRECT

subset = utils.load_dataset('TL_all_data_without_duplicates.parquet')
_ = dt.label_direction(subset, 'TL')

Loaded dataset: cleaned_data\TL_all_data_without_duplicates.parquet
6640-101753 ['up'] [ True]
6640-101755 ['up'] [ True]
6640-101813 ['up'] [ True]
6640-101814 ['up'] [ True]
6640-101831 ['up'] [ True]
6640-101874 ['up'] [ True]
6640-101875 ['up'] [ True]
6640-101952 ['up'] [ True]
6640-101988 ['up'] [ True]
6640-101992 ['up'] [ True]
6640-102009 ['up'] [ True]
6640-102028 ['up'] [ True]
6640-102053 ['up'] [ True]
6640-102092 ['up'] [ True]
6640-102117 ['up'] [ True]
6640-102118 ['up'] [ True]
6640-102119 ['up'] [ True]
6640-102120 ['up'] [ True]
6640-102121 ['up'] [ True]
6640-102122 ['up'] [ True]
6640-102123 ['up'] [ True]
6640-102138 ['up'] [ True]
6640-102139 ['up'] [ True]
6640-102140 ['up'] [ True]
6640-102148 ['up'] [ True]
6640-102149 ['up'] [ True]
6640-102150 ['up'] [ True]
6640-102151 ['up'] [ True]
6640-102152 ['up'] [ True]
6640-102263 ['up'] [ True]
6640-102423 ['up'] [ True]
6640-102453 ['up'] [ True]
6640-102557 ['up'] [ True]
6640-102685 ['up'] [ True]
6640-102705 ['

,axisnumber,id,manipulator,motorposition,speedsetpoint,timeindex,torqueactual,torquefeedforward,type,date,robot_type,robot,direction,same_direction
3390100,1,6640-101753|2021-03-01_10:37:33,TRACK_1,0.000,-0.000000,0.010,-0.015558,0.0,TL,2021-03-01,6640,6640-101753,up,True
3390101,1,6640-101753|2021-03-01_10:37:33,TRACK_1,0.000,0.380452,0.021,0.089100,0.0,TL,2021-03-01,6640,6640-101753,up,True
3390102,1,6640-101753|2021-03-01_10:37:33,TRACK_1,0.000,0.380452,0.031,0.029362,0.0,TL,2021-03-01,6640,6640-101753,up,True
3390103,1,6640-101753|2021-03-01_10:37:33,TRACK_1,0.000,-0.000000,0.040,0.125822,0.0,TL,2021-03-01,6640,6640-101753,up,True
3390104,1,6640-101753|2021-03-01_10:37:33,TRACK_1,0.000,0.760903,0.049,-0.006727,0.0,TL,2021-03-01,6640,6640-101753,up,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6491004,1,7600-100498|2022-10-13_09:22:00,TRACK_1,7.930,-20.544400,73.092,12.644300,0.0,TL,2022-10-13,7600,7600-100498,up,True
6491005,1,7600-100498|2022-10-13_09:22:00,TRACK_1,6.688,-18.071500,73.101,13.613500,0.0,TL,2022-10-13,7600,7600-100498,up,True
6491006,1,7600-100498|2022-10-13_09:22:00,TRACK_1,5.926,-17.120300,73.110,14.034900,0.0,TL,2022-10-13,7600,7600-100498,up,True
6491007,1,7600-100498|2022-10-13_09:22:00,TRACK_1,5.234,-15.979000,73.119,12.950600,0.0,TL,2022-10-13,7600,7600-100498,up,True


In [5]:
#INTERPOLATION

df = utils.load_dataset('TL_all_data_without_duplicates.parquet')
subset = df[~df['robot'].isin(['7600-100465'])] #THIS IS BECAUSE THE INTERPOLATION DOESNT WORK FOR THIS ROBOT FOR SOME REASON
print(np.isnan(subset['torqueactual']).sum())

interval = 0.01
interpolated_df = dt.interpolate_sequences(subset, interval)

dt.export_data(interpolated_df, 'interpolated_TL.parquet')

# plt.plot(interpolated_sequence['timeindex'], interpolated_sequence['torqueactual'])
# plt.show()
# plt.plot(subset[subset['id']==id]['timeindex'], subset[subset['id']==id]['torqueactual'])
# plt.show()

Loaded dataset: cleaned_data\TL_all_data_without_duplicates.parquet
0
Empty DataFrame
Columns: [id, timeindex, torqueactual]
Index: []


In [108]:
#CHECK THAT DELTA TIME IS 0.01

subset = utils.load_dataset(6)
cont = 0
ids_fault = []
ids = subset['id'].unique()
for id in ids:
    subset_id = subset[subset['id']==id]
    subset_id = subset_id.sort_values(by=['id', 'timeindex'])
    for i in range(1, len(subset_id)):
        delta_time = subset_id['timeindex'].iloc[i]-subset_id['timeindex'].iloc[i-1]
        if delta_time > 0.01 + 0.0001 or delta_time < 0.01 - 0.0001:
            # print(delta_time)
            # print(subset_id['timeindex'].iloc[i], subset_id['torqueactual'].iloc[i])
            # print(subset_id['timeindex'].iloc[i-1], subset_id['torqueactual'].iloc[i-1])
            # print('----')
            ids_fault.append(id)
    if id in ids_fault:
        cont += 1
        print(id)
print(cont)


Loaded dataset: \cleaned_data\interpolated_TL.parquet
0


In [ ]:
#FOURIER TRANSFORM

subset = utils.load_dataset(6)

print(np.isnan(subset['torqueactual']).sum())

sd = SpectrumDecomposition()
fourier_transform = sd.time_to_freq(subset)

print(np.isnan(fourier_transform['torqueactual_abs']).sum())

dt.export_data(fourier_transform, 'no_duplicates_freqs.parquet')

In [ ]:
#DROP DUPLICATES

subset = utils.load_dataset('All_raw_data.parquet', False)
subset = subset[subset['type']=='TL']
subset = dt.drop_duplicates(subset)
dt.export_data(subset, 'TL_raw_data_without_duplicates.parquet', False)

In [ ]:
#MERGE TWO DATASETS

df1 = utils.load_dataset(0, False)
df2 = utils.load_dataset(1, False)

subset = pd.concat([df1, df2])
dt.export_data(subset, 'All_raw_data.parquet', False)

In [ ]:
#PLOTTING

subset1 = utils.load_dataset('TL_raw_data_without_duplicates.parquet', False)
subset2 = utils.load_dataset('TL_all_data_without_duplicates.parquet')

y_axis_options = ['motorposition', 'speedsetpoint', 'torqueactual', 'torquefeedforward']
df_subset = subset1
df_subset2 = subset2
# df_subset = df_subset.where(df_subset['direction']=='up')
# df_subset = df_subset.dropna()
# robots = df_subset['robot'].unique()
# robot = robots[0]
# df_subset = df_subset.where(df_subset['robot']==robot)
# df_subset = df_subset.dropna()

robot_types = df_subset['robot_type'].unique()
robot_type = robot_types[1]
ids = df_subset[df_subset['robot_type']==robot_type]['id'].unique()
id = ids[10]
# id = '6640-102685|2021-06-25_04:10:24'
print(id)
series_type = 'TL'
utils.plot_graph(df_subset, id, series_type, y_axis_options[2])
utils.plot_graph(df_subset2, id, series_type, y_axis_options[2])

In [7]:
#Number of series per unique robot

subset = utils.load_dataset('TL_all_data_without_duplicates.parquet')
robot_types = subset['robot_type'].unique()
distinct_robot_types = subset.groupby(subset['id'].str[:4])['id'].nunique()
print(distinct_robot_types)
for robot_type in robot_types:
    id_subset = subset.where(subset['robot_type'] == robot_type)
    distinct_robots = id_subset.groupby(id_subset['id'].str[:11])['id'].nunique()
    print(distinct_robots)


Loaded dataset: cleaned_data\TL_all_data_without_duplicates.parquet
id
6640    237
6700     14
7600     43
Name: id, dtype: int64
id
6640-101753    6
6640-101755    5
6640-101813    6
6640-101814    6
6640-101831    6
6640-101874    6
6640-101875    5
6640-101952    6
6640-101988    4
6640-101992    5
6640-102009    3
6640-102028    6
6640-102053    6
6640-102092    6
6640-102117    3
6640-102118    5
6640-102119    6
6640-102120    5
6640-102121    6
6640-102122    5
6640-102123    5
6640-102138    5
6640-102139    5
6640-102140    6
6640-102148    6
6640-102149    6
6640-102150    6
6640-102151    5
6640-102152    5
6640-102263    3
6640-102423    6
6640-102453    3
6640-102557    3
6640-102685    3
6640-102705    3
6640-102788    3
6640-102789    3
6640-102826    3
6640-102832    3
6640-102834    2
6640-102841    3
6640-102842    3
6640-102851    6
6640-102980    4
6640-102981    3
6640-103082    5
6640-103454    4
6640-103455    5
6640-106401    3
6640-107361    1
6640-107963    5


In [6]:
#Number of unique robots per type of robot
subset = utils.load_dataset('TL_all_data_without_duplicates.parquet')
robot_types = subset['robot_type'].unique()
distinct_robot_types = subset.groupby(subset['id'].str[:4])['id'].nunique()
print(robot_types)
for robot_type in robot_types:
    id_subset = subset.where(subset['robot_type'] == robot_type)
    distinct_robots = id_subset.groupby(id_subset['id'].str[:4])['robot'].nunique()
    print(distinct_robots)

Loaded dataset: cleaned_data\TL_all_data_without_duplicates.parquet
['6640' '6700' '7600']
id
6640    52
Name: robot, dtype: int64
id
6700    5
Name: robot, dtype: int64
id
7600    3
Name: robot, dtype: int64
